In [6]:
# Set up for async background linux process runs
import os
get_ipython().system = os.system

In [2]:
!uv init

error: Project is already initialized in `/Users/asaucedo/Programming/real-time-serving` (`pyproject.toml` file exists)


In [1]:
!uv add mlserver cookiecutter scikit-learn

Resolved 164 packages in 0.29ms
Audited 160 packages in 0.13ms


In [3]:
# Run command on terminal
# > !mlserver init

#  [1/6] project_name (Lambda runtime): SimpleModel
#  [2/6] repo_user (MyGithubUsername): axsaucedo
#  [3/6] repo_name (simplemodel):
#  [4/6] class_name (Simplemodel): SimpleModel
#  [5/6] description (A short description of the project.):
#  [6/6] Select open_source_license
#    1 - MIT
#    2 - BSD-3-Clause
#    3 - No license file
#    Choose from [1/2/3] (1): 3

'/Users/asaucedo/Programming/real-time-serving/.venv/lib/python3.12/site-packages/mlserver/__init__.py'

In [4]:
!ls simplemodel/

Dockerfile           docs                 simplemodel
LICENSE              pyproject.toml       tests
Makefile             requirements-dev.txt
README.md            setup.py


In [5]:
!ls simplemodel/simplemodel/

__init__.py common.py   runtime.py  version.py


In [6]:
!bat simplemodel/simplemodel/runtime.py

───────┬────────────────────────────────────────────────────────────────────────
       │ File: simplemodel/simplemodel/runtime.py
───────┼────────────────────────────────────────────────────────────────────────
   1   │ import numpy as np
   2   │ from mlserver.model import MLModel
   3   │ from mlserver.settings import ModelSettings
   4   │ from fastapi import status
   5   │ from mlserver.utils import get_model_uri
   6   │ from mlserver.errors import InvalidModelURI, MLServerError
   7   │ from mlserver.types import (
   8   │     InferenceRequest,
   9   │     InferenceResponse,
  10   │ )
  11   │ from mlserver.codecs import NumpyCodec, NumpyRequestCodec
  12   │ from simplemodel.common import SimpleModelSettings
  13   │ 
  14   │ 
  15   │ class SimpleModel(MLModel):
  16   │     """Runtime class for specific Huggingface models"""
  17   │ 
  18   │     def __init__(self, settings: ModelSettings):
  19   │ 
  20   │         self._extra_settings = SimpleModelSettings(**settings

## Out of the Box Models
### Sklearn

In [18]:
!mkdir -p sklearn/

Training

In [2]:
# Original source code and more details can be found in:
# https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

# The digits dataset
digits = datasets.load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

SVC(gamma=0.001)

Saving the model

In [19]:
import joblib

model_file_name = "sklearn/mnist-svm.joblib"
joblib.dump(classifier, model_file_name)

['sklearn/mnist-svm.joblib']

In [23]:
!ls sklearn

mnist-svm.joblib


Serving with out of the box server

In [12]:
!uv add mlserver-sklearn

Resolved 169 packages in 626ms                                       
Prepared 1 package in 257ms                                              
Installed 1 package in 6ms.6.1                              
 + mlserver-sklearn==1.6.1


In [13]:
%%writefile sklearn/model-settings.json
{
    "name": "mnist-svm",
    "implementation": "mlserver_sklearn.SKLearnModel",
    "parameters": {
        "uri": "./mnist-svm.joblib",
        "version": "v0.1.0"
    }
}


Overwriting sklearn/model-settings.json


In [14]:
!ls sklearn/

mnist-svm.joblib
model-settings.json


0

In [20]:
!mlserver start . &

0

2024-11-09 13:04:37,066 [mlserver] WARNING - Model name 'mnist-svm' is different than model's folder name 'sklearn'.
2024-11-09 13:04:37,098 [mlserver.parallel] DEBUG - Starting response processing loop...
2024-11-09 13:04:37,099 [mlserver.rest] INFO - HTTP server running on http://0.0.0.0:8080
2024-11-09 13:04:37,111 [mlserver.metrics] INFO - Metrics server running on http://0.0.0.0:8082
2024-11-09 13:04:37,111 [mlserver.metrics] INFO - Prometheus scraping endpoint can be accessed on http://0.0.0.0:8082/metrics


INFO:     Started server process [72279]
INFO:     Waiting for application startup.
INFO:     Started server process [72279]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)


2024-11-09 13:04:37,554 [mlserver.grpc] INFO - gRPC server running on http://0.0.0.0:8081
2024-11-09 13:04:38,534 [mlserver][mnist-svm:v0.1.0] INFO - Loaded model 'mnist-svm' successfully.
2024-11-09 13:04:38,534 [mlserver][mnist-svm:v0.1.0] INFO - Loaded model 'mnist-svm' successfully.


In [28]:
import requests

x_0 = X_test[0:1]
inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": x_0.shape,
          "datatype": "FP32",
          "data": x_0.tolist()
        }
    ]
}

endpoint = "http://localhost:8080/v2/models/mnist-svm/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

print(response.json())


INFO:     127.0.0.1:58661 - "POST /v2/models/mnist-svm/versions/v0.1.0/infer HTTP/1.1" 200 OK
{'model_name': 'mnist-svm', 'model_version': 'v0.1.0', 'id': '43b121d2-035c-4ba2-9bbf-3a02d13eea12', 'parameters': {}, 'outputs': [{'name': 'predict', 'shape': [1, 1], 'datatype': 'INT64', 'parameters': {'content_type': 'np'}, 'data': [8]}]}


In [19]:
!lsof -n -i :8080

COMMAND     PID     USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME
python3.1 71850 asaucedo   81u  IPv4   0xb634458c0eb97e      0t0  TCP 127.0.0.1:58556->127.0.0.1:http-alt (CLOSED)
python3.1 71850 asaucedo   82u  IPv4 0x8b3b0df96b3d163d      0t0  TCP 127.0.0.1:58615->127.0.0.1:http-alt (CLOSE_WAIT)


0

In [18]:
!kill -9 72247

0